### Financial News Scrapper

이번 실습에서는 Google News로부터 미국 종목의 티커를 검색해 투자 조언을 생성하는 LLM을 생성하는 것으로 한다.

### 1. Google Search function

Google News에서 내용을 파싱하는 함수가 다음과 같이 존재한다. 이를 실행하기 위해서는 다음과 같은 라이브러리가 필요하다.

- `Selenium` : pip install selenium
- `webdriver_manager` : pip3 install webdriver_manager
- `Newspaper` : pip3 install newspaper3k
- `lxml_html_clean` : pip install lxml_html_clean

In [ ]:
# import package
from bs4 import BeautifulSoup
import requests
import urllib
import re
import pandas as pd
from newspaper import Article
from selenium import webdriver

In [ ]:
def search_from_google(asset:str, page_nums:int) -> pd.DataFrame :
    '''
    google news로부터 검색을 한 뒤, selenium을 통해 뉴스 데이터들을 가져옵니다.
    :param asset: 검색할 자산
    :param page_nums: 뉴스를 검색할 총 페이지의 수
    :return: news data가 들어있는 DataFrame
    '''
    keyword = f'{asset} buying reason'
    news_df = pd.DataFrame()
    
    # selenium headless mode
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    
    for page_num in range(0, page_nums-1):
        # google news crawling
        url = f'https://www.google.com/search?q={keyword}&sca_esv=1814fa2a4600643d&tbas=0&tbs=qdr:m&tbm=nws&ei=rE3pZeLxNeHX1e8PpdOcMA&start={page_num}&sa=N&ved=2ahUKEwji9-zrsuGEAxXha_UHHaUpBwYQ8tMDegQIBBAE&biw=2560&bih=1313&dpr=1'
        req = requests.get(url)
        content = req.content
        soup = BeautifulSoup(content, 'html.parser')
    
        # last page check
        if soup.select('div.BNeawe.vvjwJb') == []: break
    
        title_list = [t.text for t in soup.select('div.BNeawe.vvjwJb')]  # title
        url_list = []
    
        # url
        for u in soup.select('a'):
            for t in title_list:
                if t in u.text:
                    temp_url = urllib.parse.unquote(u['href'])
                    temp_url = re.findall('http\S+&sa',temp_url)[0][:-3]
                    url_list.append(temp_url)
    
        # article
        for ind, news_url in enumerate(url_list):
            try:
                article = Article(url=news_url)
                article.download()
                article.parse()    
                news_article = article.text
            except:  # ssl error
                driver.get(news_url)
                article.download(input_html=driver.page_source)
                article.parse()
                news_article = article.text
    
            news_df = pd.concat([news_df, pd.DataFrame([[title_list[ind], news_article, news_url]])])
    
        news_df[0] = news_df[0].apply(lambda x: re.sub('\s+',' ',x))
        news_df = news_df.reset_index(drop=True)
    
    news_df.columns = ['Title','Contents','URL']
    
    return news_df

In [ ]:
search_result = search_from_google('META', 3)

In [ ]:
# 출력 결과의 확인
search_result.head()

### 2. Prompt Template

1번에서 수집된 뉴스 데이터의 투자 포인트와 매수, 매도를 결정하는 함수 `get_investing_point_llm`과 `get_side_from_gpt`를 아래의 규칙에 따라서 정의한다.

#### `get_investing_point_llm`
- 복잡한 과업을 하위 과업으로 분할하여 지시한다.
- 산출한 결과에 대해서 정당화 요청을 수행한다.
- 여러 개의 결과를 출력해 가장 적합한 것을 선택하도록 한다.
- 지침을 반복한다.
- 구분선을 사용하여 예시를 제공한다.
- 만약 기사 내용이 기업과 관련이 없는 경우, '관련 없음'의 내용이 채워지도록 지시한다.

#### `get_side_from_gpt`

- 지침을 반복함으로써, Sell, Buy, 혹은 Hold로 세 개의 label만 나오도록 강제한다.

혹은, 1번의 기능과 결합하여 독립적인 Class로 디자인하여도 무방하다.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
import os
import openai

CONFIG_PATH = '' # write your api key path

# with open("../config/open_ai.key", "r") as f:
#     lines = f.readlines()
#     config = lines[0].strip()
# 
# openai.api_key = config
# os.environ["OPENAI_API_KEY"] = config

In [ ]:
def get_investing_point_llm(
        asset:str,
        news_df: pd.DataFrame,
        temperature:float,
        lm_model:str,
    ) -> pd.DataFrame :
    """
    뉴스가 포함된 데이터프레임으로부터 새로운 열 ['investing_point']를 생성하여 GPT로부터 생성된 투자 포인트를 저장하는 함수입니다.
    :param asset: 자산의 티커입니다.
    :param news_df: 1번으로부터 생성된 데이터프레임을 넣으면 됩니다.
    :param temperature: LLM Model의 temperature를 지정하는 parameter입니다.
    :param lm_model: 사용할 LLM Model을 지정하는 parameter입니다.
    :return: news_df에 투자 포인트의 컬럼이 추가된 pd.DataFrame형태의 data frame입니다.
    """
    pass

def get_side_from_gpt(
        news_df: pd.DataFrame,
        temperature:float,
        lm_model:str,
    ) -> pd.DataFrame :
    """
    뉴스가 포함된 데이터프레임으로부터 새로운 열 ['investing_point']를 사용하여 GPT로부터 생성된 투자 방향(position)을 생성하여 저장하는 함수입니다.
    :param news_df: get_investing_point_llm 함수로부터 출력된 결과물을 넣습니다.
    :param temperature: LLM Model의 temperature를 지정하는 parameter입니다.
    :param lm_model: 사용할 LLM Model을 지정하는 parameter입니다.
    :return: news_df에 side의 컬럼이 추가된 pd.DataFrame형태의 data frame입니다.
    """
    pass

### 3. Agent GPT

2번으로부터 생성된 투자 포인트를 결합하여 다음의 과업을 수행하는 일련의 Agent 모델을 생성한다

- 생성된 투자포인트의 모음으로부터 실제로 발생한 사실과 의견을 구분하는 Agent를 생성한다.
- 향후 전망을 전문적으로 수행하는 에이전트를 생성해 기업 전망을 생성하도록 한다.
- side로부터 정보를 불러와, 최종적으로 매수와 매도를 판단하는 Agent를 생성한다.

아래의 규칙에 따라 Agent를 생성하고 결과물을 출력해 보자.

- 산출한 결과에 대해서 정당화 요청을 수행한다.
- 여러 개의 결과를 출력해 가장 적합한 것을 선택하도록 한다.
- 번역한 내용 자체는 출력하지 않도록 반복 요청한다.
- 사실을 출력하기 위한 Agent는 예시를 제공하여 (ex: eps가 3% 시장 컨센서스를 상회하였음) 올바른 답을 내도록 유도한다.

### 4. ReAct

3번으로부터 보고받은 결과물에 대해, `SerpAPI`를 사용하여 발생한 사실들에 대해 예상되는 효과를 검색하여 추론하도록 설계한다. 

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

SERP_API_KEY = '<KEY>' # enter your SerpAPI Key!

chat = ChatOpenAI(
    model_name = 'gpt-4o-mini',
    api_key = openai.api_key,
    
)
# tools = load_tools(
#     ['serpapi'],
#     serpapi_api_key = SERP_API_KEY
# )
# prompt = hub.pull('hwchase17/react')